In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from plotly.offline import iplot
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 11})

import warnings 
warnings.simplefilter('ignore')
import pickle

with open('../Имена_скважин_pickle/dict_name_well.pickle', 'rb') as f:
    dict_name_well = pickle.load(f)
with open('../Имена_скважин_pickle/dict_name_well_more_365.pickle', 'rb') as f:
    dict_name_well_more_365 = pickle.load(f)
with open('../Имена_скважин_pickle/dict_name_well_reverse.pickle', 'rb') as f:
    dict_name_well_reverse = pickle.load(f)
with open('../Имена_скважин_pickle/dict_name_well_oil_more_365.pickle', 'rb') as f:
    dict_name_well_oil_more_365 = pickle.load(f)

%config InlineBackend.figure_formats = 'svg'

from functions.make_df_without_gaps import make_df_without_gaps
from functions.make_df_with_count_days_ago import make_df_with_count_days_ago
from functions.uppend_p_zab import uppend_p_zab

In [2]:
gtm = pd.read_csv('../../data/Исходные_данные/gtm.csv')
gtm['WELL_NAME'] = gtm['WELL_NAME'].map(dict_name_well_oil_more_365)
gtm.dropna(inplace=True)
gtm.reset_index(inplace=True, drop=True)

In [3]:
gtm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   WELL_NAME        225 non-null    float64
 1   GEOTECH_ACTION   225 non-null    object 
 2   START_DATE_FACT  225 non-null    object 
 3   END_DATE_FACT    225 non-null    object 
dtypes: float64(1), object(3)
memory usage: 7.2+ KB


In [4]:
train_oil_year_accum = pd.read_csv('../../data/preprocessed_data/train_oil_year_accum.csv')
train_oil_year_accum.head()

,MEASURED_IN_DATE,WELL_NAME,CHARWORK,WATER_CUT,P_ZAB,INTAKE,WELLHEAD_PRESSURE,LIQ_RATE,OIL_RATE,WATER_RATE,Q_LIQ_ACCUM,Q_OIL_ACCUM,Q_WATER_ACCUM
0,2020-01-17,1,НЕФ,26.0,59.20,0.0,NaN,221.0,163.54,57.46,221.0,163.54,57.46
1,2020-01-18,1,НЕФ,26.0,59.20,0.0,NaN,221.0,163.54,57.46,442.0,327.08,114.92
2,2020-01-19,1,НЕФ,26.0,59.20,0.0,NaN,221.0,163.54,57.46,663.0,490.62,172.38
3,2020-01-20,1,НЕФ,26.0,59.20,0.0,NaN,221.0,163.54,57.46,884.0,654.16,229.84
4,2020-01-21,1,НЕФ,30.0,59.26,0.0,NaN,208.0,145.60,62.40,1092.0,799.76,292.24


In [5]:
data_measured = train_oil_year_accum[['MEASURED_IN_DATE', 'WELL_NAME']]
data_measured['MEASURED_IN_DATE'] = pd.to_datetime(data_measured['MEASURED_IN_DATE'])

df_measured = pd.DataFrame()

for well in data_measured['WELL_NAME'].unique():
    data = pd.DataFrame()
    start_date = data_measured[data_measured['WELL_NAME'] == well]['MEASURED_IN_DATE'].min()
    end_date = data_measured[data_measured['WELL_NAME'] == well]['MEASURED_IN_DATE'].max()
    
    data['MEASURED_IN_DATE'] = pd.date_range(start=start_date, 
                                             end=end_date, 
                                             freq='D')
    data['WELL_NAME'] = well
    data = data[['WELL_NAME', 'MEASURED_IN_DATE']]
    df_measured = pd.concat([df_measured, data])
df_measured.reset_index(drop=True, inplace=True)
df_measured

,WELL_NAME,MEASURED_IN_DATE
0,1,2020-01-17
1,1,2020-01-18
2,1,2020-01-19
3,1,2020-01-20
4,1,2020-01-21
...,...,...
125367,258,2022-05-27
125368,258,2022-05-28
125369,258,2022-05-29
125370,258,2022-05-30


In [6]:
data_gtm = gtm[['WELL_NAME', 'START_DATE_FACT', 'END_DATE_FACT']]
data_gtm['START_DATE_FACT'] = pd.to_datetime(data_gtm['START_DATE_FACT'])
data_gtm['END_DATE_FACT'] = pd.DataFrame(data_gtm['END_DATE_FACT'])

df_gtm = pd.DataFrame()

for i in range(data_gtm.shape[0]):
    data = pd.DataFrame()
    well = data_gtm['WELL_NAME'].iloc[i]
    start_date, end_date = data_gtm['START_DATE_FACT'].iloc[i], data_gtm['END_DATE_FACT'].iloc[i]
    

    data['GTM_IN_DATE'] = pd.date_range(start=start_date, 
                                        end=end_date, 
                                        freq='D')
    data['WELL_NAME'] = well
    data = data[['WELL_NAME', 'GTM_IN_DATE']]
    df_gtm = pd.concat([df_gtm, data])
     
df_gtm.reset_index(drop=True, inplace=True)    
df_gtm

,WELL_NAME,GTM_IN_DATE
0,148.0,2020-10-03
1,148.0,2020-10-04
2,148.0,2020-10-05
3,148.0,2020-10-06
4,235.0,2020-07-01
...,...,...
2722,37.0,2016-12-13
2723,37.0,2016-12-14
2724,37.0,2016-12-15
2725,37.0,2016-12-16


In [7]:
data_without_gaps = make_df_without_gaps(dataframe=train_oil_year_accum, 
                                         date_column='MEASURED_IN_DATE', 
                                         group_params='WELL_NAME')

data_count_days_ago = make_df_with_count_days_ago(dataframe=data_without_gaps, 
                            count_days=5, 
                            target='OIL_RATE')
data_pressure = uppend_p_zab(data_count_days_ago, train_oil_year_accum)
data_pressure

,MEASURED_IN_DATE,P_ZAB,WELL_NAME,OIL_RATE_5_days_ago,OIL_RATE_4_days_ago,OIL_RATE_3_days_ago,OIL_RATE_2_days_ago,OIL_RATE_1_days_ago,TARGET_OIL_RATE
0,2020-09-05,51.60,1,98.80,97.60,99.63,96.40,91.20,90.82
1,2020-09-06,50.59,1,97.60,99.63,96.40,91.20,90.82,92.82
2,2020-09-07,50.55,1,99.63,96.40,91.20,90.82,92.82,96.76
3,2020-09-08,50.55,1,96.40,91.20,90.82,92.82,96.76,96.76
4,2020-09-09,53.55,1,91.20,90.82,92.82,96.76,96.76,95.94
...,...,...,...,...,...,...,...,...,...
42218,2021-09-20,49.83,258,99.36,99.82,100.28,101.20,100.74,99.82
42219,2021-09-21,48.92,258,99.82,100.28,101.20,100.74,99.82,101.66
42220,2021-09-22,48.92,258,100.28,101.20,100.74,99.82,101.66,100.28
42221,2021-09-23,48.92,258,101.20,100.74,99.82,101.66,100.28,99.36


In [8]:
df_gtm.rename(columns={'GTM_IN_DATE': 'DATE'}, inplace=True)
df_measured.rename(columns={'MEASURED_IN_DATE': 'DATE'}, inplace=True)


merge_df = pd.merge(df_measured, df_gtm, on=['WELL_NAME', 'DATE'], how='outer', indicator=True)
merge_df[merge_df['_merge'] == 'both']

,WELL_NAME,DATE,_merge
0,1.0,2020-01-17,both
1338,4.0,2019-10-09,both
1339,4.0,2019-10-10,both
1340,4.0,2019-10-11,both
1341,4.0,2019-10-12,both
...,...,...,...
121024,249.0,2021-03-31,both
121648,250.0,2020-11-04,both
121649,250.0,2020-11-05,both
121650,250.0,2020-11-06,both


In [40]:
data = pd.read_csv('../../data/preprocessed_data/dates_intersection_of_gtm_and_measured.csv')
df = pd.read_csv('../../data/preprocessed_data/train_last_50_days_ago.csv')


date_gtm_and_measured = data[data['_merge'] == 'both'][['DATE', 'WELL_NAME']]
date_gtm_and_measured.rename(columns={'DATE': 'MEASURED_IN_DATE'}, inplace=True)


df_well_and_measured = df[[ 'MEASURED_IN_DATE', 'WELL_NAME']]

df_well_and_measured.merge(date_gtm_and_measured, on=['MEASURED_IN_DATE', 'WELL_NAME'], how='inner')

,MEASURED_IN_DATE,WELL_NAME
0,2020-06-15,4
1,2021-03-20,76
2,2021-09-13,195
3,2021-05-02,237
4,2021-03-27,249


In [50]:
df[df['WELL_NAME'] == 249][['MEASURED_IN_DATE', 'TARGET_OIL_RATE']]

,MEASURED_IN_DATE,TARGET_OIL_RATE
34101,2020-10-10,46.15
34102,2020-10-11,45.50
34103,2020-10-12,48.10
34104,2020-10-13,44.20
34105,2020-10-14,44.85
...,...,...
34265,2021-03-23,109.62
34266,2021-03-24,110.49
34267,2021-03-25,110.08
34268,2021-03-26,110.08


In [49]:
gtm[(gtm['WELL_NAME'] == 237)]

,WELL_NAME,GEOTECH_ACTION,START_DATE_FACT,END_DATE_FACT
58,237.0,Из бурения,2020-08-11,2020-08-14
177,237.0,ИДН,2021-05-02,2021-05-05
204,237.0,ИДН,2022-01-05,2022-01-09
205,237.0,ИДН,2022-07-10,2022-07-13
